## <font color=#0099CC>mIAx - Taller Renta Fija - ANÁLISIS CARTERA DE RENTA FIJA</font>

En esta práctica, desarrollaremos un análisis relativamente exahustivo de un universo de Renta Fija, en concreto, de bonos corporativos. Además, construiremos y analizaremos varias carteras. 

Para ello, contaremos con la siguiente información almacenada en la carpeta *data*:
- Universo de bonos, con sus características esenciales (fichero *universo.csv*)
- Histórico de precios de cierre del universo de bonos anterior (fichero *precios_historicos_universo.csv*)
- Curva de tipos de interés €STR (fichero *curvaESTR.csv*)
- Histórico de precios de otros índices que nos serán de utilidad (fichero *precios_historicos_varios*):
    - Índices de crédito: ITRAXX Main y ITRAXX XOVER. Serán útiles para la cobertura del riesgo de crédito.
    - Futuros sobre el *Schatz* (DU1), *BOBL* (OE1) y *BUND* (RX1). Serán útiles para la cobertura de los tipos de interés.
    - Índice de crédito *RECMTREU*, que valdría como benchmark de las carteras que construyamos.

No necesariamente se usará toda toda la información

En líneas generales, estos son los ejercicios que completaremos, aunque los detallaremos más en cada apartado:
1. Análisis de datos. En esta sección, haremos un análisis de la información que tenemos de cada bono y lo que significa. Asimismo, haremos los tratamientos y limpieza que necesitemos para luego poder usarlos.
2. Valoración de los bonos del universo utilizando la curva de descuento y bajo ciertas asunciones. Comparación de estos precios con los precios de mercado.
3. Cálculo del spread que pagan los bonos sobre la curva.
4. Cálculo de *yield*, duración y convexidad.
5. Contrucción de una cartera equiponderada con todos los bonos del universo. Contraste con el benchmark (os proponemos el índice RECMTREU para el que os hemos dado los precios) y backtest de la estrategia. ¡OJO! El índice es *Total Return*.
6. Tienes el mandato de construir una cartera de como máximo **20** bonos corporativos con ese universo y una serie de restricciones y, claro, maximizando la rentabilidad total de la cartera:
    - La duración de la cartera no debe superar los 3 años
    - La exposición a emisiones HY no puede superar el 10% de la cartera
    - No puedes invertir en deuda subordinada
    - No se puede invertir en emisiones de tamaño igual o inferior a 500 millones
    - No se puede invertir más de un 10% del capital en una misma emisión
    - No puede haber más de un 15% de concentración en un mismo emisor
    (¡OJO! No estamos teniendo en cuenta en este ejercicio si hubiera un mínimo de inversión, lo cuál sería un dato relevante tener en cuenta en un caso real)

    6.1. Constrúyela a fecha de hoy

    6.2. Teniendo en cuenta la naturaleza que nos están pidiendo para la cartera, ¿añadirías alguna otra restricción?

    6.3. ¿Cómo medirías el riesgo de crédito de la cartera?

    6.4. ¿Cómo medirías el riesgo de liquidez de la cartera?

    6.5. Describe cómo habría que hacer el backtest de esta cartera
7. Ahora, se te pide que cubras la exposición de la cartera a los tipos de interés. Con la información que tienes, ¿cómo lo harías?
8. ¿Y si quisieras cubrir total o parcialmente el riesgo de crédito? Usa de nuevo la información que tienes.
9. ¿Cómo construirías tu cartera? ¿Se te ocurre alguna estrategia específica, por ejemplo, de valor relativo?


📣 <font color=#CC6600>**¡NORMAS!**</font>

La práctica se puede hacer en grupos de hasta **3 personas** y deberá entregarse antes del **27 de noviembre**. 

Cada grupo expondrá una parte de los ejercicios en la clase del día 27, donde la resolveremos juntos a modo de taller. Esta exposición contará hasta **1 punto** de la nota final.

Además, se valorará positivamente para la práctica la participación en las clases.

📣 <font color=#CC6600>**¡IMPORTANTE!**</font>

Todo el código implementado debe estar debidamente comentado e incluir conclusiones de los resultados obtenidos para optar a la máxima puntuación. Asimismo, se debe responder a las preguntas planteadas.

Las propuestas de mejora o posibles trabajos futuros se valorarán positivamente también.

Usa las fuentes de información que consideres necesarias para apoyar tus respuestas.

### <font color=#336699>Librerías</font>

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import datetime

In [2]:
# Estilo
plt.style.use('dark_background')

In [3]:
# Fecha de análisis
fecha_analisis = pd.to_datetime('2025-10-01')

### <font color=#336699>1. Datos</font>

<style>.gray {background-color: #595959}

</style><div class="gray">❕💬¿Qué observas en los datos? Analiza la información que tenemos del universo</div>

Haz un primer análisis visual de la información que tenemos del universo de bono.
Entre otras, plantéate cuestiones como:
- ¿Divisas?
- ¿Tipo de bonos? ¿Fijo/Flotante? ¿Prelación? ¿Opcionalidad? ¿Hay bonos perpetuos?
- ¿Sectores? ¿Emisores? Si invirtéramos en todos los bonos, ¿dirías a priori que la cartera está diversificada?
- ¿Ratings? (Riesgo de crédito)
- ¿Otros datos cuantitativos?
    - Riesgo de liquidez - Horquillas y nominal vivo
- ¿Hay *gaps* en la información que vamos a tener que tratar?

In [ ]:
# Cargamos el universo de bonos
universo = pd.read_csv('../data/universo.csv', sep=';')

# Información básica del dataset
print(f"Total de bonos en el universo: {len(universo)}")
print(f"\nColumnas disponibles: {list(universo.columns)}")
print(f"\nPrimeras filas:")
universo.head()


#### <font color=#808080>Divisas</font>

In [ ]:
# Análisis de divisas
divisas = universo['Ccy'].value_counts()

print("Distribución de divisas:")
print(divisas)
print(f"\nTotal de divisas diferentes: {len(divisas)}")

# Visualización
fig, ax = plt.subplots(figsize=(10, 6))
divisas.plot(kind='bar', ax=ax, color='cyan')
ax.set_title('Distribución de Bonos por Divisa', fontsize=14, fontweight='bold')
ax.set_xlabel('Divisa', fontsize=12)
ax.set_ylabel('Número de Bonos', fontsize=12)
ax.tick_params(axis='x', rotation=0)
plt.tight_layout()
plt.show()

# Porcentaje del total
print(f"\nPorcentaje por divisa:")
for ccy, count in divisas.items():
    pct = (count / len(universo)) * 100
    print(f"{ccy}: {pct:.2f}%")


#### <font color=#808080>Tipos de bono</font>

In [ ]:
# Análisis de tipos de bono

# 1. Tipo de cupón (Fijo vs Variable)
tipo_cupon = universo['Coupon Type'].value_counts()
print("Distribución por tipo de cupón:")
print(tipo_cupon)
print(f"\nPorcentaje:")
for tipo, count in tipo_cupon.items():
    pct = (count / len(universo)) * 100
    print(f"{tipo}: {pct:.2f}%")

# 2. Prelación (Seniority)
prelacion = universo['Seniority'].value_counts()
print(f"\n\nDistribución por prelación:")
print(prelacion)
print(f"\nPorcentaje:")
for sen, count in prelacion.items():
    pct = (count / len(universo)) * 100
    print(f"{sen}: {pct:.2f}%")

# 3. Opcionalidad (Callable)
callable_bonds = universo['Callable'].value_counts()
print(f"\n\nBonos con opción de compra (Callable):")
print(callable_bonds)
print(f"\nPorcentaje:")
for call, count in callable_bonds.items():
    pct = (count / len(universo)) * 100
    print(f"{call}: {pct:.2f}%")

# 4. Bonos perpetuos (sin fecha de vencimiento)
perpetuos = universo[universo['Maturity'].isna() | (universo['Maturity'] == '')]
print(f"\n\nBonos perpetuos (sin fecha de vencimiento): {len(perpetuos)}")
if len(perpetuos) > 0:
    print("\nEjemplos de bonos perpetuos:")
    print(perpetuos[['ISIN', 'Description', 'Issuer', 'Coupon Type', 'Next Call Date']].head())

# Visualizaciones
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Tipo de cupón
tipo_cupon.plot(kind='bar', ax=axes[0, 0], color='cyan')
axes[0, 0].set_title('Distribución por Tipo de Cupón', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Tipo de Cupón')
axes[0, 0].set_ylabel('Número de Bonos')
axes[0, 0].tick_params(axis='x', rotation=45)

# Prelación
prelacion.plot(kind='bar', ax=axes[0, 1], color='orange')
axes[0, 1].set_title('Distribución por Prelación', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Prelación')
axes[0, 1].set_ylabel('Número de Bonos')
axes[0, 1].tick_params(axis='x', rotation=45)

# Callable
callable_bonds.plot(kind='bar', ax=axes[1, 0], color='green')
axes[1, 0].set_title('Bonos Callable vs No Callable', fontsize=12, fontweight='bold')
axes[1, 0].set_xlabel('Callable')
axes[1, 0].set_ylabel('Número de Bonos')
axes[1, 0].tick_params(axis='x', rotation=0)

# Frecuencia de cupón
freq_cupon = universo['Coupon Frequency'].value_counts()
freq_cupon.plot(kind='bar', ax=axes[1, 1], color='magenta')
axes[1, 1].set_title('Distribución por Frecuencia de Cupón', fontsize=12, fontweight='bold')
axes[1, 1].set_xlabel('Frecuencia de Cupón')
axes[1, 1].set_ylabel('Número de Bonos')
axes[1, 1].tick_params(axis='x', rotation=0)

plt.tight_layout()
plt.show()


#### <font color=#808080>Sectores y emisores</font>

In [ ]:
# Análisis de sectores y emisores

# 1. Distribución por sectores
sectores = universo['Industry Sector'].value_counts()
print("Distribución por sectores:")
print(sectores)
print(f"\nTotal de sectores diferentes: {len(sectores)}")

# 2. Distribución por emisores
emisores = universo['Issuer'].value_counts()
print(f"\n\nTotal de emisores diferentes: {len(emisores)}")
print(f"\nTop 10 emisores por número de emisiones:")
print(emisores.head(10))

# 3. Análisis de concentración
# Si invirtiéramos en todos los bonos, ¿estaría diversificada la cartera?
# Calculamos el peso de cada emisor si la cartera fuera equiponderada
peso_por_emisor = emisores / len(universo) * 100
concentracion_top10 = peso_por_emisor.head(10).sum()
print(f"\n\nConcentración de los top 10 emisores: {concentracion_top10:.2f}%")
print(f"Concentración del top emisor: {peso_por_emisor.iloc[0]:.2f}%")

# Visualizaciones
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Sectores - Top 15
sectores.head(15).plot(kind='barh', ax=axes[0, 0], color='cyan')
axes[0, 0].set_title('Top 15 Sectores por Número de Bonos', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Número de Bonos')
axes[0, 0].invert_yaxis()

# Emisores - Top 15
emisores.head(15).plot(kind='barh', ax=axes[0, 1], color='orange')
axes[0, 1].set_title('Top 15 Emisores por Número de Emisiones', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Número de Emisiones')
axes[0, 1].invert_yaxis()

# Concentración de emisores (distribución acumulada)
emisores_sorted = emisores.sort_values(ascending=False)
concentracion_acum = (emisores_sorted.cumsum() / len(universo) * 100).head(50)
axes[1, 0].plot(range(1, len(concentracion_acum) + 1), concentracion_acum.values, marker='o', color='green')
axes[1, 0].set_title('Concentración Acumulada de Emisores (Top 50)', fontsize=12, fontweight='bold')
axes[1, 0].set_xlabel('Número de Emisores (ordenados)')
axes[1, 0].set_ylabel('Porcentaje Acumulado de Bonos (%)')
axes[1, 0].grid(True, alpha=0.3)
axes[1, 0].axhline(y=50, color='r', linestyle='--', label='50%')
axes[1, 0].legend()

# Distribución de número de emisiones por emisor
num_emisiones_por_emisor = emisores.value_counts().sort_index()
axes[1, 1].bar(num_emisiones_por_emisor.index, num_emisiones_por_emisor.values, color='magenta')
axes[1, 1].set_title('Distribución: Número de Emisores por Cantidad de Emisiones', fontsize=12, fontweight='bold')
axes[1, 1].set_xlabel('Número de Emisiones')
axes[1, 1].set_ylabel('Número de Emisores')
axes[1, 1].set_yscale('log')

plt.tight_layout()
plt.show()

# Conclusión sobre diversificación
print(f"\n{'='*60}")
print("CONCLUSIÓN SOBRE DIVERSIFICACIÓN:")
print(f"{'='*60}")
print(f"Si invirtiéramos en todos los bonos del universo:")
print(f"- Tendríamos exposición a {len(emisores)} emisores diferentes")
print(f"- Tendríamos exposición a {len(sectores)} sectores diferentes")
print(f"- El emisor más concentrado representaría {peso_por_emisor.iloc[0]:.2f}% de la cartera")
print(f"- Los top 10 emisores representarían {concentracion_top10:.2f}% de la cartera")
if concentracion_top10 > 30:
    print(f"- ⚠️ La cartera presenta cierta concentración en los top emisores")
else:
    print(f"- ✓ La cartera está relativamente diversificada por emisores")


#### <font color=#808080>Ratings (Riesgo de crédito)</font>

In [ ]:
# Análisis de Ratings (Riesgo de crédito)

# Distribución de ratings
ratings = universo['Rating'].value_counts().sort_index()
print("Distribución de ratings:")
print(ratings)
print(f"\nTotal de ratings diferentes: {len(ratings)}")

# Clasificación Investment Grade vs High Yield
# Investment Grade: AAA, AA+, AA, AA-, A+, A, A-, BBB+, BBB, BBB-
# High Yield: BB+, BB, BB-, B+, B, B-, CCC+, CCC, CCC-, CC, C, D
investment_grade = ['AAA', 'AA+', 'AA', 'AA-', 'A+', 'A', 'A-', 'BBB+', 'BBB', 'BBB-']
high_yield = ['BB+', 'BB', 'BB-', 'B+', 'B', 'B-', 'CCC+', 'CCC', 'CCC-', 'CC', 'C', 'D']

def clasificar_rating(rating):
    if pd.isna(rating) or rating == '' or rating == 'NR':
        return 'No Rating'
    rating_clean = str(rating).strip()
    if rating_clean in investment_grade:
        return 'Investment Grade'
    elif rating_clean in high_yield:
        return 'High Yield'
    else:
        return 'No Rating'

universo['Rating_Class'] = universo['Rating'].apply(clasificar_rating)
rating_class = universo['Rating_Class'].value_counts()

print(f"\n\nClasificación por categoría de riesgo:")
print(rating_class)
print(f"\nPorcentaje:")
for clase, count in rating_class.items():
    pct = (count / len(universo)) * 100
    print(f"{clase}: {pct:.2f}%")

# Análisis de PD (Probabilidad de Default a 1 año)
print(f"\n\nAnálisis de Probabilidad de Default (PD 1YR):")
print(f"Valores nulos: {universo['PD 1YR'].isna().sum()}")
print(f"Estadísticas descriptivas:")
print(universo['PD 1YR'].describe())

# Visualizaciones
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Distribución de ratings
ratings.plot(kind='bar', ax=axes[0, 0], color='cyan')
axes[0, 0].set_title('Distribución de Ratings', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Rating')
axes[0, 0].set_ylabel('Número de Bonos')
axes[0, 0].tick_params(axis='x', rotation=45)

# Clasificación IG vs HY
rating_class.plot(kind='bar', ax=axes[0, 1], color=['green', 'red', 'gray'])
axes[0, 1].set_title('Distribución Investment Grade vs High Yield', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Categoría')
axes[0, 1].set_ylabel('Número de Bonos')
axes[0, 1].tick_params(axis='x', rotation=45)

# Distribución de PD
pd_data = universo['PD 1YR'].dropna()
axes[1, 0].hist(pd_data, bins=50, color='orange', edgecolor='black', alpha=0.7)
axes[1, 0].set_title('Distribución de Probabilidad de Default (PD 1YR)', fontsize=12, fontweight='bold')
axes[1, 0].set_xlabel('PD 1YR')
axes[1, 0].set_ylabel('Frecuencia')
axes[1, 0].set_yscale('log')
axes[1, 0].grid(True, alpha=0.3)

# Box plot de PD por Rating Class
universo_pd = universo[universo['PD 1YR'].notna()]
if len(universo_pd) > 0:
    rating_classes = universo_pd['Rating_Class'].unique()
    data_to_plot = [universo_pd[universo_pd['Rating_Class'] == rc]['PD 1YR'].values 
                    for rc in rating_classes if len(universo_pd[universo_pd['Rating_Class'] == rc]) > 0]
    axes[1, 1].boxplot(data_to_plot, labels=[rc for rc in rating_classes if len(universo_pd[universo_pd['Rating_Class'] == rc]) > 0])
    axes[1, 1].set_title('PD 1YR por Categoría de Rating', fontsize=12, fontweight='bold')
    axes[1, 1].set_ylabel('PD 1YR')
    axes[1, 1].set_yscale('log')
    axes[1, 1].tick_params(axis='x', rotation=45)
    axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


#### <font color=#808080>Riesgo de liquidez - Horquillas y nominal vivo</font>

In [ ]:
# Análisis de Riesgo de Liquidez - Horquillas y Nominal Vivo

# 1. Cálculo de horquillas (Bid-Ask Spread)
universo['Bid_Ask_Spread'] = universo['Ask Price'] - universo['Bid Price']
universo['Bid_Ask_Spread_Pct'] = (universo['Bid_Ask_Spread'] / universo['Price']) * 100

print("Análisis de Horquillas (Bid-Ask Spread):")
print(f"Valores nulos en Bid Price: {universo['Bid Price'].isna().sum()}")
print(f"Valores nulos en Ask Price: {universo['Ask Price'].isna().sum()}")
print(f"\nEstadísticas del spread absoluto:")
print(universo['Bid_Ask_Spread'].describe())
print(f"\nEstadísticas del spread porcentual:")
print(universo['Bid_Ask_Spread_Pct'].describe())

# 2. Análisis de Nominal Vivo (Outstanding Amount)
print(f"\n\nAnálisis de Nominal Vivo (Outstanding Amount):")
print(f"Valores nulos: {universo['Outstanding Amount'].isna().sum()}")
print(f"Estadísticas descriptivas (en millones):")
outstanding_millions = universo['Outstanding Amount'] / 1_000_000
print(outstanding_millions.describe())

# Clasificación por tamaño de emisión
def clasificar_tamano(nominal):
    if pd.isna(nominal):
        return 'Sin dato'
    nominal_mill = nominal / 1_000_000
    if nominal_mill < 500:
        return '< 500M'
    elif nominal_mill < 1000:
        return '500M - 1B'
    elif nominal_mill < 2000:
        return '1B - 2B'
    else:
        return '> 2B'

universo['Tamaño_Emision'] = universo['Outstanding Amount'].apply(clasificar_tamano)
tamano_dist = universo['Tamaño_Emision'].value_counts()
print(f"\n\nDistribución por tamaño de emisión:")
print(tamano_dist)

# Relación entre horquilla y tamaño de emisión
print(f"\n\nCorrelación entre horquilla y tamaño de emisión:")
correlacion = universo[['Bid_Ask_Spread_Pct', 'Outstanding Amount']].corr()
print(correlacion)

# Visualizaciones
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# Histograma de horquillas absolutas
spread_data = universo['Bid_Ask_Spread'].dropna()
axes[0, 0].hist(spread_data, bins=50, color='cyan', edgecolor='black', alpha=0.7)
axes[0, 0].set_title('Distribución de Horquillas (Bid-Ask Spread Absoluto)', fontsize=11, fontweight='bold')
axes[0, 0].set_xlabel('Spread (Precio)')
axes[0, 0].set_ylabel('Frecuencia')
axes[0, 0].grid(True, alpha=0.3)

# Histograma de horquillas porcentuales
spread_pct_data = universo['Bid_Ask_Spread_Pct'].dropna()
axes[0, 1].hist(spread_pct_data, bins=50, color='orange', edgecolor='black', alpha=0.7)
axes[0, 1].set_title('Distribución de Horquillas (Bid-Ask Spread %)', fontsize=11, fontweight='bold')
axes[0, 1].set_xlabel('Spread (%)')
axes[0, 1].set_ylabel('Frecuencia')
axes[0, 1].grid(True, alpha=0.3)

# Distribución de nominal vivo
outstanding_data = outstanding_millions.dropna()
axes[0, 2].hist(outstanding_data, bins=50, color='green', edgecolor='black', alpha=0.7)
axes[0, 2].set_title('Distribución de Nominal Vivo (Millones)', fontsize=11, fontweight='bold')
axes[0, 2].set_xlabel('Nominal Vivo (Millones)')
axes[0, 2].set_ylabel('Frecuencia')
axes[0, 2].set_yscale('log')
axes[0, 2].grid(True, alpha=0.3)

# Box plot de horquilla por tamaño de emisión
tamano_categories = universo['Tamaño_Emision'].unique()
data_to_plot = [universo[universo['Tamaño_Emision'] == tam]['Bid_Ask_Spread_Pct'].dropna().values 
                for tam in tamano_categories if len(universo[universo['Tamaño_Emision'] == tam]['Bid_Ask_Spread_Pct'].dropna()) > 0]
labels_to_plot = [tam for tam in tamano_categories if len(universo[universo['Tamaño_Emision'] == tam]['Bid_Ask_Spread_Pct'].dropna()) > 0]
if len(data_to_plot) > 0:
    axes[1, 0].boxplot(data_to_plot, labels=labels_to_plot)
    axes[1, 0].set_title('Horquilla por Tamaño de Emisión', fontsize=11, fontweight='bold')
    axes[1, 0].set_ylabel('Spread (%)')
    axes[1, 0].tick_params(axis='x', rotation=45)
    axes[1, 0].grid(True, alpha=0.3)

# Scatter: Horquilla vs Nominal Vivo
universo_clean = universo[['Bid_Ask_Spread_Pct', 'Outstanding Amount']].dropna()
if len(universo_clean) > 0:
    axes[1, 1].scatter(universo_clean['Outstanding Amount'] / 1_000_000, 
                       universo_clean['Bid_Ask_Spread_Pct'], 
                       alpha=0.5, color='magenta', s=10)
    axes[1, 1].set_title('Horquilla vs Nominal Vivo', fontsize=11, fontweight='bold')
    axes[1, 1].set_xlabel('Nominal Vivo (Millones)')
    axes[1, 1].set_ylabel('Spread (%)')
    axes[1, 1].set_xscale('log')
    axes[1, 1].grid(True, alpha=0.3)

# Distribución por tamaño de emisión
tamano_dist.plot(kind='bar', ax=axes[1, 2], color='red')
axes[1, 2].set_title('Distribución por Tamaño de Emisión', fontsize=11, fontweight='bold')
axes[1, 2].set_xlabel('Tamaño de Emisión')
axes[1, 2].set_ylabel('Número de Bonos')
axes[1, 2].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# Conclusiones
print(f"\n{'='*60}")
print("CONCLUSIONES SOBRE RIESGO DE LIQUIDEZ:")
print(f"{'='*60}")
print(f"- Horquilla media: {universo['Bid_Ask_Spread_Pct'].mean():.3f}%")
print(f"- Horquilla mediana: {universo['Bid_Ask_Spread_Pct'].median():.3f}%")
print(f"- Bonos con horquilla > 1%: {(universo['Bid_Ask_Spread_Pct'] > 1).sum()} ({(universo['Bid_Ask_Spread_Pct'] > 1).sum()/len(universo)*100:.2f}%)")
print(f"- Bonos con emisión < 500M: {(outstanding_millions < 500).sum()} ({(outstanding_millions < 500).sum()/len(universo)*100:.2f}%)")
print(f"- Bonos con emisión >= 500M: {(outstanding_millions >= 500).sum()} ({(outstanding_millions >= 500).sum()/len(universo)*100:.2f}%)")


In [ ]:
# Análisis de gaps en la información

print("="*60)
print("ANÁLISIS DE GAPS EN LA INFORMACIÓN")
print("="*60)

# Resumen de valores nulos por columna
print("\n1. VALORES NULOS EN EL UNIVERSO:")
print("-" * 60)
nulos_universo = universo.isnull().sum()
nulos_universo_pct = (nulos_universo / len(universo)) * 100
gaps_df = pd.DataFrame({
    'Valores_Nulos': nulos_universo,
    'Porcentaje': nulos_universo_pct
})
gaps_df = gaps_df[gaps_df['Valores_Nulos'] > 0].sort_values('Valores_Nulos', ascending=False)
print(gaps_df)

# Análisis específico de campos críticos
print("\n\n2. ANÁLISIS DE CAMPOS CRÍTICOS:")
print("-" * 60)
campos_criticos = {
    'Maturity': 'Fecha de vencimiento (crítica para valoración)',
    'Coupon': 'Cupón (crítica para valoración)',
    'Rating': 'Rating (importante para riesgo de crédito)',
    'Outstanding Amount': 'Nominal vivo (importante para liquidez)',
    'Bid Price': 'Precio bid (importante para liquidez)',
    'Ask Price': 'Precio ask (importante para liquidez)',
    'Next Call Date': 'Fecha call (importante si es callable)',
    'PD 1YR': 'Probabilidad de default (importante para riesgo)'
}

for campo, descripcion in campos_criticos.items():
    if campo in universo.columns:
        nulos = universo[campo].isnull().sum()
        pct = (nulos / len(universo)) * 100
        print(f"\n{campo}:")
        print(f"  Descripción: {descripcion}")
        print(f"  Valores nulos: {nulos} ({pct:.2f}%)")
        if campo == 'Callable':
            # Verificamos si hay bonos callable sin fecha call
            callable_sin_fecha = ((universo['Callable'] == 'Y') & 
                                  (universo['Next Call Date'].isna() | (universo['Next Call Date'] == '')))
            if callable_sin_fecha.sum() > 0:
                print(f"  ⚠️ Bonos callable sin fecha call: {callable_sin_fecha.sum()}")

# Bonos con información incompleta crítica
print("\n\n3. BONOS CON INFORMACIÓN INCOMPLETA CRÍTICA:")
print("-" * 60)
bonos_incompletos = universo[
    universo['Maturity'].isna() | 
    universo['Coupon'].isna() |
    (universo['Price'].isna())
]
print(f"Bonos sin fecha de vencimiento, cupón o precio: {len(bonos_incompletos)}")
if len(bonos_incompletos) > 0:
    print("\nEjemplos:")
    print(bonos_incompletos[['ISIN', 'Description', 'Maturity', 'Coupon', 'Price']].head(10))

# Verificación de consistencia de datos
print("\n\n4. VERIFICACIONES DE CONSISTENCIA:")
print("-" * 60)

# Precios bid > ask (inconsistencia)
if 'Bid Price' in universo.columns and 'Ask Price' in universo.columns:
    bid_ask_inconsistente = universo[
        (universo['Bid Price'].notna()) & 
        (universo['Ask Price'].notna()) &
        (universo['Bid Price'] > universo['Ask Price'])
    ]
    print(f"Bonos con Bid Price > Ask Price (inconsistencia): {len(bid_ask_inconsistente)}")

# Precio fuera del rango bid-ask
if 'Price' in universo.columns and 'Bid Price' in universo.columns and 'Ask Price' in universo.columns:
    precio_fuera_rango = universo[
        (universo['Price'].notna()) &
        (universo['Bid Price'].notna()) & 
        (universo['Ask Price'].notna()) &
        ((universo['Price'] < universo['Bid Price']) | (universo['Price'] > universo['Ask Price']))
    ]
    print(f"Bonos con Price fuera del rango Bid-Ask: {len(precio_fuera_rango)}")

# Bonos callable sin fecha call
callable_sin_fecha = ((universo['Callable'] == 'Y') & 
                      (universo['Next Call Date'].isna() | (universo['Next Call Date'] == '')))
print(f"Bonos Callable sin fecha de call: {len(callable_sin_fecha)}")

# Bonos con cupón 0 o negativo
if 'Coupon' in universo.columns:
    cupon_anomalo = universo[(universo['Coupon'] <= 0) & universo['Coupon'].notna()]
    print(f"Bonos con cupón <= 0: {len(cupon_anomalo)}")

# Resumen final
print("\n\n5. RESUMEN Y RECOMENDACIONES:")
print("-" * 60)
print(f"Total de bonos en el universo: {len(universo)}")
print(f"Bonos con información completa crítica: {len(universo) - len(bonos_incompletos)}")
print(f"Bonos que podrían necesitar tratamiento especial: {len(bonos_incompletos)}")

if len(bonos_incompletos) > 0:
    print("\n⚠️ RECOMENDACIONES:")
    print("- Revisar bonos sin fecha de vencimiento (posiblemente perpetuos)")
    print("- Verificar bonos sin cupón o precio")
    print("- Considerar excluir o tratar especialmente bonos con información crítica faltante")


#### <font color=#808080>Resto de información</font>

En esta sección, analizamos el resto de ficheros para ver qué información tenemos y, en caso de haber *gaps*, limpiar los datos antes de trabajar con ellos.

##### <font color=#CC6600>Precios bonos universo</font>

In [ ]:
# Cargamos los precios históricos del universo
precios_universo = pd.read_csv('../data/precios_historicos_universo.csv', sep=';')

print("Análisis de Precios Históricos del Universo:")
print(f"Shape: {precios_universo.shape}")
print(f"\nColumnas: {list(precios_universo.columns)}")
print(f"\nPrimeras filas:")
print(precios_universo.head())

print(f"\n\nInformación sobre fechas:")
if 'Date' in precios_universo.columns:
    precios_universo['Date'] = pd.to_datetime(precios_universo['Date'])
    print(f"Rango de fechas: {precios_universo['Date'].min()} a {precios_universo['Date'].max()}")
    print(f"Total de fechas únicas: {precios_universo['Date'].nunique()}")

print(f"\n\nValores nulos por columna:")
print(precios_universo.isnull().sum())

# Verificamos si hay ISINs que coincidan con nuestro universo
if 'ISIN' in precios_universo.columns:
    isins_precios = set(precios_universo['ISIN'].unique())
    isins_universo = set(universo['ISIN'].unique())
    print(f"\n\nComparación de ISINs:")
    print(f"ISINs en precios históricos: {len(isins_precios)}")
    print(f"ISINs en universo: {len(isins_universo)}")
    print(f"ISINs comunes: {len(isins_precios & isins_universo)}")
    print(f"ISINs solo en precios: {len(isins_precios - isins_universo)}")
    print(f"ISINs solo en universo: {len(isins_universo - isins_precios)}")


##### <font color=#CC6600>Otros precios</font>

Para terminar con el análisis de datos, falta lo leído en los ficheros de *"precios_historicos_varios.csv"* y *curvaESTR.csv*.

In [ ]:
# Cargamos los otros precios históricos y la curva
precios_varios = pd.read_csv('../data/precios_historicos_varios.csv', sep=';')
curva_estr = pd.read_csv('../data/curvaESTR.csv', sep=';')

print("="*60)
print("ANÁLISIS DE PRECIOS HISTÓRICOS VARIOS")
print("="*60)
print(f"Shape: {precios_varios.shape}")
print(f"\nColumnas: {list(precios_varios.columns)}")
print(f"\nPrimeras filas:")
print(precios_varios.head())

if 'Date' in precios_varios.columns:
    precios_varios['Date'] = pd.to_datetime(precios_varios['Date'])
    print(f"\n\nRango de fechas: {precios_varios['Date'].min()} a {precios_varios['Date'].max()}")
    print(f"Total de fechas únicas: {precios_varios['Date'].nunique()}")

# Identificamos qué instrumentos tenemos
if 'Ticker' in precios_varios.columns or 'Instrument' in precios_varios.columns:
    col_ticker = 'Ticker' if 'Ticker' in precios_varios.columns else 'Instrument'
    instrumentos = precios_varios[col_ticker].unique()
    print(f"\n\nInstrumentos disponibles:")
    for inst in instrumentos:
        print(f"  - {inst}")

print(f"\n\nValores nulos por columna:")
print(precios_varios.isnull().sum())

print("\n" + "="*60)
print("ANÁLISIS DE CURVA ESTR")
print("="*60)
print(f"Shape: {curva_estr.shape}")
print(f"\nColumnas: {list(curva_estr.columns)}")
print(f"\nPrimeras filas:")
print(curva_estr.head())

if 'Date' in curva_estr.columns:
    curva_estr['Date'] = pd.to_datetime(curva_estr['Date'])
    print(f"\n\nRango de fechas: {curva_estr['Date'].min()} a {curva_estr['Date'].max()}")
    print(f"Total de fechas únicas: {curva_estr['Date'].nunique()}")

print(f"\n\nValores nulos por columna:")
print(curva_estr.isnull().sum())

# Visualización de la curva para la fecha de análisis
if fecha_analisis in curva_estr['Date'].values:
    curva_fecha = curva_estr[curva_estr['Date'] == fecha_analisis].copy()
    if len(curva_fecha) > 0 and 'Zero Rate' in curva_fecha.columns:
        fig, ax = plt.subplots(figsize=(12, 6))
        ax.plot(curva_fecha['Date'], curva_fecha['Zero Rate'], marker='o', color='cyan', linewidth=2)
        ax.set_title(f'Curva ESTR - {fecha_analisis.strftime("%Y-%m-%d")}', fontsize=14, fontweight='bold')
        ax.set_xlabel('Fecha')
        ax.set_ylabel('Zero Rate (%)')
        ax.grid(True, alpha=0.3)
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()


### <font color=#336699>2. Valoración</font>

<style>.gray {background-color: #595959}

</style><div class="gray">

❕💬 En esta sección, valoraremos los bonos utilizando la curva. Para ello, crea una función (puedes hacerlo en un .py aparte) que con las **características del bono, la curva y un spread de crédito** devuelva la valoración del bono (incluyendo **precio limpio, cupón corrido y precio sucio**).

Si asumimos que el **spread de crédito es 0**, y la ejecutamos para el 01/10/2025...
- ¿Qué observas si comparas los precios obtenidos y los precios de mercado? 
- ¿Crees que la diferencia se debe a un factor relacionado sólo con el riesgo crediticio?
- ¿Qué otros factores influyen en ese spread?

Para la valoración, haz las siguientes simplificaciones:

- Asume que el vencimiento de los bonos perpetuos (para los que no hay vencimiento) es la próxima fecha call.
- Asume que todos aquellos bonos que tengan call serán calleados. Por lo tanto, usa la fecha call como fecha de vencimiento.
- Asume que los cupones son fijos hasta vencimiento (aunque alguno cambie a lo largo de la vida del bono).
- Usa la base de cálculo ACT/365. No tengas en cuenta la convención de día hábil.

Ten en cuenta que necesitarás una función de interpolación también. Interpola los factores de descuento exponencialmente.

</div>

In [53]:
# Establecemos la fecha
fecha_valor = datetime.datetime(2025,10,1)

### <font color=#336699>3. Spread</font>

<style>.gray {background-color: #595959}

</style><div class="gray">

❕💬 Calculemos ahora los spreads que debemos añadir a la curva con un movimiento paralelo para que cuadren los precios de mercado que tenemos. Para ello, usa la función de valoración del apartado anterior. 
- ¿Qué observas? ¿Tienen sentido los resultados? 
- ¿Con qué datos de los que tenemos compararías para ver si los resultados son coherentes?

</div>

### <font color=#336699>4. YTM, Duración, Convexidad</font>

<style>.gray {background-color: #595959}

</style><div class="gray">

❕💬 Calculemos ahora la siguiente información, también relacionada con la rentabilidad y riesgo de las emisiones:
- *Yield* - Por simplicidad, en este caso, en el caso de los bonos callable, nos quedaremos con la fecha call, como en el ejercicio anterior. Usa las mismas asunciones que para la valoración y el spread.
- Duración
- Convexidad

Responde a las siguientes preguntas:
- ¿Que relación hay entre la TIR calculada y el spread calculado en el apartado anterior?
- ¿Qué relación hay entre la duración y el vencimiento? ¿Qué refleja la duración? ¿De qué otra forma se podría obtener esta sensibilidad?
- Estima el precio del bono usando la duración y convexidad, ¿qué observas?

</div>

### <font color=#336699>5. Cartera equiponderada</font>

<style>.gray {background-color: #595959}

</style><div class="gray">

❕💬 Crea un algoritmo de inversión que consista en una cartera equiponderada, asignando el mismo peso a todos los bonos vivos en cada fecha de rebalanceo.

Asume rebalanceo mensual, y que no tenemos costes más allá de los implícitos en el propio precio, calcula la evolución que hubiese tenido tu algoritmo. Por simplificación, utiliza los precios MID que se te dan.

Asumiendo que el benchmark de la cartera es el índice que se nos da: *RECMTREU Index*. Contrasta la evolución de tú cartera contra dicho benchmark. Ten cuidado porque es un índice *Total Return*.

- ¿Qué sería lo más correcto en lugar de utilizar los precios MID?
- ¿Se te ocurre algún otro benchmark que se podría utilizar?

</div>

### <font color=#336699>6. Cartera mandato</font>

<style>.gray {background-color: #595959}

</style><div class="gray">

❕💬 Como adelantábamos en el enunciado, tienes el mandato de construir una cartera de como máximo **20** bonos corporativos con ese universo y una serie de restricciones y, claro, maximizando la rentabilidad total de la cartera:
- La duración de la cartera no debe superar los 3 años
- La exposición a emisiones HY no puede superar el 10% de la cartera
- No puedes invertir en deuda subordinada
- No se puede invertir en emisiones de tamaño igual o inferior a 500 millones
- No se puede invertir más de un 10% del capital en una misma emisión
- No puede haber más de un 15% de concentración en un mismo emisor
(¡OJO! No estamos teniendo en cuenta en este ejercicio si hubiera un mínimo de inversión, lo cuál sería un dato relevante tener en cuenta en un caso real)

1. Teniendo en cuenta la naturaleza que nos están pidiendo para la cartera, ¿añadirías alguna otra restricción?

2. ¿Cómo medirías el riesgo de crédito de la cartera?

3. ¿Cómo medirías el riesgo de liquidez de la cartera? ¿Se te ocurre alguna otra información que se podría utilizar aunque no se te haya dado?

4. Describe cómo habría que hacer el backtest de esta cartera, no hace falta que lo implementes en este caso

</div>

##### <font color=#CC6600>Riesgo de crédito</font>

##### <font color=#CC6600>Riesgo de liquidez</font>

##### <font color=#CC6600>Backtest</font>

### <font color=#336699>7. Cobertura tipos de interés</font>

<style>.gray {background-color: #595959}

</style><div class="gray">

❕💬 Utiliza alguno de los siguientes instrumentos de los que te hemos dado para cubrir la duración (sensibilidad de tipos de interés) de la cartera que has construido según el mandato. Asume una inversión en la cartera de 10 millones:

- Futuros sobre el *Schatz* (ticker: DU1) - Duración a 01/10/2025: 1.92
- Futuros sobre el *BOBL* (ticker: OE1) - Duración a 01/10/2025: 5.44
- Futuros sobre el *BUND* (ticker: RX1) - Duración a 01/10/2025: 10

*Contract size* en todos los casos: 100,000 euros

Investiga sobre estos instrumentos antes de tomar la decisión. Razona tu elección del instrumento y el número de contratos que has decidido comprar/vender.

- ¿Qué pasaría si comprásemos/vendiésemos 100 futuros?
- ¿Se te ocurre algún otro instrumento con el que cubrir la sensibilidad a los tipos de interés de la cartera?

</div>

### <font color=#336699>8. Cobertura crédito</font>

<style>.gray {background-color: #595959}

</style><div class="gray">

❕💬 Utiliza alguno de los siguientes instrumentos de los que te hemos dado para cubrir el riesgo de crédito de la cartera que has construido según el mandato. Asume una inversión en la cartera de 10 millones:

- ITRAXX Main (ticker: ITRX EUR CDSI GEN 5Y Corp)
- ITRAXX XOVER (ticker: ITRX XOVER CDSI GEN 5Y Corp)

Estos índices cotizan en forma de spread, en puntos básicos. La sensibilidad del valor del swap (CDS) la vamos a asumir en 4,500€ al punto básico asumiendo una inversión de 10 millones.

Investiga sobre estos instrumentos antes de tomar la decisión. Razona tu elección del instrumento y el nominal que has decidido comprar/vender.

- ¿Tiene sentido plantear esta cobertura total?
- ¿Con qué otros instrumentos podrías cubrir el riesgo de crédito?

</div>

### <font color=#336699>9. Estrategia propia</font>

<style>.gray {background-color: #595959}

</style><div class="gray">

❕💬 Plantea tu propia estrategia con la información que tienes. Puede ser una estrategia direccional, de valor relativo, que hayas visto o no en clase; pero siempre razonando tu planteamiento.

</div>